## Section 0. Environmental Setup

### Step A. Check Tensorflow Version

In [12]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
print("Tensorflow Version: ", tf.__version__)

Tensorflow Version:  2.18.0


### Step B. Check GPU Availability and Set VRAM Growth


In [16]:
print("Numbers of GPU Available: ", len(tf.config.list_physical_devices('GPU')))


for gpu in tf.config.list_physical_devices('GPU'):
    print("Name:", gpu.name, " ; Type:", gpu.device_type)
    tf.config.experimental.set_memory_growth(gpu, True)
    print("Dynamic memory growth enabled.")

Numbers of GPU Available:  1
Name: /physical_device:GPU:0  ; Type: GPU
Dynamic memory growth enabled.
